In [31]:
from model_server.model_server.data_loader import DataLoader
from model_server.model_server.data_cleaner import DataCleaner
from model_server.model_server.feature_extractor import FeatureExtractor
from model_server.model_server.model import Model

In [32]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='model_server/model_server/.env')

True

In [33]:
data_loader = DataLoader(entsoe_api_key=os.getenv('ENTSOE_API_KEY'))
data_loader.update_df(out_df_filepath="data/bronze/df.parquet")

In [34]:
DataCleaner.clean(
    in_df_filepath='data/bronze/df.parquet',
    out_df_filepath='data/silver/df.parquet',
)

In [35]:
FeatureExtractor.extract_features(
    in_df_filepath='data/silver/df.parquet',
    out_df_filepath='data/gold/df.parquet',
)

In [36]:
model = Model(model_filepath='data/model.joblib')
model.train(Xy_filepath='data/gold/df.parquet', n_estimators=100)

In [37]:
_, mape = model.backtest(
    Xy_filepath='data/gold/df.parquet',
    starting_ts=pd.Timestamp(datetime.now() - pd.Timedelta(30, 'd'), tz='Europe/Zurich'),
    use_every_nth_ts=100,
)
print(f'Backtested MAPE: {mape:.2f}%')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.20it/s]

Backtested MAPE: 5.04%


In [39]:
yhat = model.predict(
    in_df_filepath='data/gold/df.parquet', 
    out_yhat_filepath='data/yhat.parquet',
)
yhat

,predicted_24h_later_load
2024-09-27 19:00:00+02:00,6474.173239
2024-09-27 20:00:00+02:00,6224.201867
2024-09-27 21:00:00+02:00,6062.327307
2024-09-27 22:00:00+02:00,6147.052759
2024-09-27 23:00:00+02:00,6333.368725
2024-09-28 00:00:00+02:00,6214.023296
2024-09-28 01:00:00+02:00,6042.244918
2024-09-28 02:00:00+02:00,6306.203754
2024-09-28 03:00:00+02:00,6188.901375
2024-09-28 04:00:00+02:00,5614.965050
